<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings

warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
# with open('./data/hopsworks-api-key.txt', 'r', encoding='utf-8') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read()

project = hopsworks.login()
fs = project.get_feature_store()
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "france", "city": "paris", "street": "18eme", "aqicn_url": "https://api.waqi.info/feed/@3082", "latitude": 48.86, "longitude": 2.32}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [4]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

pm25 country   city street       date                               url
0   1.0  france  paris  18eme 2024-11-19  https://api.waqi.info/feed/@3082

In [5]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [6]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 48.75°N 2.25°E
Elevation 46.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


date  temperature_2m_mean  precipitation_sum  wind_speed_10m_max  \
0 2024-11-19                11.70                0.7           25.460936   
1 2024-11-20                 7.00                0.0           16.622490   
2 2024-11-21                 2.05                0.5           11.620809   
3 2024-11-22                 2.20                0.2           13.044724   
4 2024-11-23                 4.55                0.0           19.161337   
5 2024-11-24                18.15                0.0           29.215502   
6 2024-11-25                11.80                0.6           10.105681   
7 2024-11-26                10.15                0.0           13.627795   
8 2024-11-27                 5.75                0.0            8.427383   
9 2024-11-28                 5.00                0.2           12.646200   

   wind_direction_10m_dominant   city  
0                   236.534592  paris  
1                   287.650208  paris  
2                    73.810730  paris  
3                   242.020615  paris  
4                   154.397858  paris  
5                   202.456985  paris  
6                   265.914459  paris  
7                   257.799561  paris  
8                    19.983198  paris  
9                     4.899013  paris

In [7]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


In [8]:
df_aq = air_quality_fg.read()
df_aq.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.02s) 


date  pm25 country   city street  \
0 2024-01-20 00:00:00+00:00  66.0  france  paris  18eme   
1 2022-06-19 00:00:00+00:00  57.0  france  paris  18eme   
2 2024-04-19 00:00:00+00:00  23.0  france  paris  18eme   
3 2024-08-13 00:00:00+00:00  38.0  france  paris  18eme   
4 2022-11-25 00:00:00+00:00  51.0  france  paris  18eme   

                                url  pm25_3days  
0  https://api.waqi.info/feed/@3082   67.333333  
1  https://api.waqi.info/feed/@3082   66.000000  
2  https://api.waqi.info/feed/@3082   61.500000  
3  https://api.waqi.info/feed/@3082   48.666667  
4  https://api.waqi.info/feed/@3082   39.333333

In [9]:
df_aq = df_aq._append(aq_today_df)
df_aq['pm25_3days'] = df_aq['pm25'].shift(1).rolling(window=3, min_periods=1).mean()
df_aq = df_aq.dropna(subset=['pm25_3days'])
df_aq.head()

date  pm25 country   city street  \
1  2022-06-19 00:00:00+00:00  57.0  france  paris  18eme   
2  2024-04-19 00:00:00+00:00  23.0  france  paris  18eme   
3  2024-08-13 00:00:00+00:00  38.0  france  paris  18eme   
4  2022-11-25 00:00:00+00:00  51.0  france  paris  18eme   
5  2021-12-23 00:00:00+00:00  87.0  france  paris  18eme   

                                url  pm25_3days  
1  https://api.waqi.info/feed/@3082   66.000000  
2  https://api.waqi.info/feed/@3082   61.500000  
3  https://api.waqi.info/feed/@3082   48.666667  
4  https://api.waqi.info/feed/@3082   39.333333  
5  https://api.waqi.info/feed/@3082   37.333333

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(df_aq)

2024-11-19 21:37:29,731 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1149079/fs/1139782/fg/1355097


Uploading Dataframe: 0.00% |          | Rows 0/1642 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1149079/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x20c57570d60>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 678928
         }
       },
       "result": {
         "observed_value": 1.0,
         "element_count": 1642,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T08:37:29.000731Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "un

In [11]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-19 21:37:39,655 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1149079/fs/1139782/fg/1354152


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1149079/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x20c57421a00>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677994
         }
       },
       "result": {
         "observed_value": 8.427383422851562,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T08:37:39.000654Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
